In [ ]:
%%bigquery
LOAD DATA INTO `qwiklabs-gcp-01-4b0fd1e23a0f.weather_data.weather_data_raw`
FROM FILES (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv']
)

Query is running:   0%|          |

""


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `weather_data.weather_data_model` REMOTE
WITH CONNECTION DEFAULT OPTIONS (ENDPOINT = 'gemini-2.0-flash');

Query is running:   0%|          |

""


In [3]:
%%bigquery
SELECT ml_generate_text_result['candidates'][0]['content']["parts"][0]["text"] AS generated_text, *
FROM
  ML.GENERATE_TEXT(
    MODEL `weather_data.weather_data_model`,
    (
      SELECT
        CONCAT('Create a weather report based on the following information: Terperature in F = ', temperature_f, 'Wind Speed in MPH = ', wind_speed_mph, 'Precipitation in Inches = ', precipitation_in, 'Barometric Pressure in Hg = ', barometric_pressure_inHg, 'Humidity % = ', humidity_percent, 'Weather Condition = ', weather_condition) AS prompt,
        date, city, state
      FROM
        `weather_data.weather_data_raw`
    ),
    STRUCT(
      0.2 AS temperature,
      1000 AS max_output_tokens));

Query is running:   0%|          |

Downloading:   0%|          |

,generated_text,ml_generate_text_result,ml_generate_text_status,prompt,date,city,state
0,"""Okay, here's a weather report based on the pr...","{""candidates"":[{""avg_logprobs"":-0.196346437608...",,Create a weather report based on the following...,2025-02-19,Atlanta,GA
1,"""Alright, here's your weather report:\n\n**Cur...","{""candidates"":[{""avg_logprobs"":-0.175018930027...",,Create a weather report based on the following...,2025-03-03,Atlanta,GA
2,"""Okay, here's a weather report based on the pr...","{""candidates"":[{""avg_logprobs"":-0.143446323483...",,Create a weather report based on the following...,2025-02-24,Atlanta,GA
3,"""Okay, here's a weather report based on the pr...","{""candidates"":[{""avg_logprobs"":-0.270186049214...",,Create a weather report based on the following...,2025-03-12,Atlanta,GA
4,"""Okay, here's a weather report based on the pr...","{""candidates"":[{""avg_logprobs"":-0.188164273897...",,Create a weather report based on the following...,2025-03-13,Atlanta,GA
...,...,...,...,...,...,...,...
295,"""Here's a weather report based on the provided...","{""candidates"":[{""avg_logprobs"":-0.156790354895...",,Create a weather report based on the following...,2025-02-21,Seattle,WA
296,"""Here's a weather report based on the provided...","{""candidates"":[{""avg_logprobs"":-0.197508516367...",,Create a weather report based on the following...,2025-02-28,Seattle,WA
297,"""Here's a weather report based on the provided...","{""candidates"":[{""avg_logprobs"":-0.173293274957...",,Create a weather report based on the following...,2025-03-16,Seattle,WA
298,"""Okay, here's a weather report based on the pr...","{""candidates"":[{""avg_logprobs"":-0.215738296508...",,Create a weather report based on the following...,2025-03-07,Seattle,WA


In [9]:
%%bigquery
CREATE OR REPLACE TABLE `weather_data.weather_data_raw_with_results` AS(
SELECT TO_JSON_STRING(ml_generate_text_result['candidates'][0]['content']["parts"][0]["text"]) AS generated_text, *
FROM
  ML.GENERATE_TEXT(
    MODEL `weather_data.weather_data_model`,
    (
      SELECT
      CONCAT('Create a weather report based on the following information: Terperature in F = ', temperature_f, 'Wind Speed in MPH = ', wind_speed_mph, 'Precipitation in Inches = ', precipitation_in, 'Barometric Pressure in Hg = ', barometric_pressure_inHg, 'Humidity % = ', humidity_percent, 'Weather Condition = ', weather_condition, '. return a value that is a string without newlines and in a format that can be loaded into BigQuery. Limit response to only vital information and keep to 20 words or less') AS prompt,
        *
      FROM
        `weather_data.weather_data_raw`
    ),
    STRUCT(
      0.2 AS temperature,
      1000 AS max_output_tokens)));

Query is running:   0%|          |

""
